In [10]:
from scripts import network, differentialEvolution, objFunction
import numpy as np
import datetime

### Check objective function: supply-demand ratio

In [22]:
net = network.WaterDistributionNetwork('ctown_pd.inp')
net.set_time_params(duration=3600, hydraulic_step=300)

In [23]:
net.run()

In [24]:
df_links = net.df_links_report
df_nodes = net.df_nodes_report

In [25]:
df_links

link            pumps                                                      \
id                PU1                          PU2                    PU3   
properties       flow     energy status       flow     energy status flow   
00:00:00    96.590064  43.065440    1.0  96.609061  43.060132    1.0  0.0   
00:05:00    96.599741  43.062737    1.0  96.618741  43.057425    1.0  0.0   
00:10:00    96.609324  43.060059    1.0  96.628326  43.054743    1.0  0.0   
00:15:00    96.618863  43.057391    1.0  96.637868  43.052071    1.0  0.0   
00:20:00    96.628358  43.054734    1.0  96.647365  43.049410    1.0  0.0   
00:25:00    96.637805  43.052089    1.0  96.656815  43.046761    1.0  0.0   
00:30:00    96.647198  43.049457    1.0  96.666211  43.044125    1.0  0.0   
00:35:00    96.656543  43.046837    1.0  96.675559  43.041502    1.0  0.0   
00:40:00    96.665841  43.044229    1.0  96.684860  43.038889    1.0  0.0   
00:45:00    96.675096  43.041632    1.0  96.694117  43.036288    1.0  0.0   
00:50:00    96.684299  43.039047    1.0  96.703323  43.033700    1.0  0.0   
00:55:00    96.693455  43.036474    1.0  96.712482  43.031123    1.0  0.0   
00:58:33    96.699920  43.034657    1.0  96.718949  43.029303    1.0  0.0   
01:00:00    96.139140  43.189559    1.0  96.158004  43.184439    1.0  0.0   

link                                 ... valves                             \
id                              PU4  ...     v1       V45                    
properties energy status       flow  ... status  velocity      flow status   
00:00:00      0.0    0.0  33.925675  ...    1.0  0.132768  2.421893    1.0   
00:05:00      0.0    0.0  33.915281  ...    1.0  0.132767  2.421875    1.0   
00:10:00      0.0    0.0  33.904866  ...    1.0  0.132767  2.421875    1.0   
00:15:00      0.0    0.0  33.894426  ...    1.0  0.132767  2.421875    1.0   
00:20:00      0.0    0.0  33.883961  ...    1.0  0.132767  2.421875    1.0   
00:25:00      0.0    0.0  33.873472  ...    1.0  0.132767  2.421875    1.0   
00:30:00      0.0    0.0  33.862959  ...    1.0  0.132767  2.421875    1.0   
00:35:00      0.0    0.0  33.852422  ...    1.0  0.132767  2.421875    1.0   
00:40:00      0.0    0.0  33.841860  ...    1.0  0.132767  2.421875    1.0   
00:45:00      0.0    0.0  33.831275  ...    1.0  0.132767  2.421875    1.0   
00:50:00      0.0    0.0  33.820666  ...    1.0  0.132767  2.421875    1.0   
00:55:00      0.0    0.0  33.810032  ...    1.0  0.132767  2.421875    1.0   
00:58:33      0.0    0.0  33.802466  ...    1.0  0.132767  2.421875    1.0   
01:00:00      0.0    0.0  33.354401  ...    1.0  0.097458  1.777796    1.0   

link                                                                
id               V47                         V2                     
properties  velocity      flow status  velocity        flow status  
00:00:00    0.281040  2.278495    1.0  2.063351  104.552098    1.0  
00:05:00    0.281038  2.278476    1.0  2.060275  104.396247    1.0  
00:10:00    0.281038  2.278476    1.0  2.057212  104.241037    1.0  
00:15:00    0.281038  2.278476    1.0  2.054161  104.086449    1.0  
00:20:00    0.281038  2.278476    1.0  2.051123  103.932489    1.0  
00:25:00    0.281038  2.278476    1.0  2.048097  103.779161    1.0  
00:30:00    0.281038  2.278476    1.0  2.045083  103.626465    1.0  
00:35:00    0.281038  2.278476    1.0  2.042082  103.474390    1.0  
00:40:00    0.281038  2.278476    1.0  2.039094  103.322953    1.0  
00:45:00    0.281038  2.278476    1.0  2.036117  103.172133    1.0  
00:50:00    0.281038  2.278476    1.0  2.033153  103.021944    1.0  
00:55:00    0.281038  2.278476    1.0  2.030201  102.872379    1.0  
00:58:33    0.281038  2.278476    1.0  2.028114  102.766594    1.0  
01:00:00    0.206299  1.672544    1.0  2.057531  104.257180    1.0  

[14 rows x 45 columns]

In [27]:
objFunction.supply_demand_ratio(net, ['PU1','PU2'])

2137.4297444176477
2705.357330064271


1.2657058493407267

In [28]:
objFunction.supply_demand_ratio2(net, ['PU1','PU2'])

1.265705849340726

In [29]:
total_demand = 0
for junc_id in net.junctions.uid:
    total_demand += df_nodes['junctions', junc_id, 'demand'].sum()
total_demand

2137.4297444176486

New method: sum of the ratios at each timestamp

In [21]:
tot_r = 0
tot_d = 0 
tot_ratio = 0

for time in df_links.index:
    demand = sum([df_nodes.loc[time]['junctions', junc_id, 'demand'] for junc_id in net.junctions.uid])
    flow = sum([df_links.loc[time]['pumps', pump_id, 'flow'] for pump_id in net.pumps.uid])
    ratio = flow/demand
    print(ratio)
    tot_ratio += ratio
tot_ratio

0.47321270587247943
0.4732270868317517
0.9999996777331489
0.9999996782999285
0.9999996770153096
0.9999996778224257
0.9999996777497266
0.999999677803064
0.9999996779088709
0.9999996776148276
0.9999996769747629
0.9999996769589696
0.9999996779063294
0.9999996780084267
0.9999996783057898


13.946435602805812

### Differential Evolution

In [6]:
 DE_params = {
        'vars_list': ['78', '79'],
        #'vars_list': ['PU1', 'PU2', 'PU3', 'PU4', 'PU5', 'PU6', 'PU7', 'PU8', 'PU9', 'PU10', 'PU11'],
        'n_pop': 20,
        'n_generations': 5,
        'CR': 0.7,
        'F': 0.5
    }

In [7]:
net = network.WaterDistributionNetwork("anytown.inp")
de = differentialEvolution.DifferentialEvolution(net, objFunction.supply_demand_ratio, **DE_params)

In [8]:
duration = 86400           # 1 day
hyd_step = 600             # 10 min
update_interval = 14400    # 4 hours

In [9]:
de.run(duration, hyd_step, update_interval)

> STARTING GENERATION 0: ...


C:\Users\david\Documents\PyCharmProjects\msc_thesis\epynet\epanet2.py:677: UserWarning: WARNING: System has negative pressures.
  warnings.warn(self.ENgeterror(ierr))


>>> Parent: candidate = {'78': array([0, 1, 0, 0, 0, 0]), '79': array([0, 1, 0, 0, 1, 0])}, value = 0.3287670048776069
>>> Child: candidate = {'78': array([0, 1, 0, 1, 0, 0]), '79': array([0, 0, 1, 0, 1, 1])}, value = 0.8219174810068427
    -> NEW BEST CANDIDATE FOUND!
    --> Substitution: True
 ----- Elapsed time [0:00:20.077887] -----
>>> Parent: candidate = {'78': array([1, 1, 0, 0, 0, 1]), '79': array([1, 1, 1, 1, 1, 0])}, value = 0.9928325562774437
>>> Child: candidate = {'78': array([1, 0, 0, 0, 0, 0]), '79': array([0, 1, 0, 0, 1, 0])}, value = 0.49953839428358343
    -> NEW BEST CANDIDATE FOUND!
    --> Substitution: False
 ----- Elapsed time [0:00:39.700970] -----
>>> Parent: candidate = {'78': array([1, 0, 1, 0, 0, 0]), '79': array([1, 1, 1, 1, 1, 0])}, value = 0.829567315245396


KeyboardInterrupt: 